In [1]:
ROOT_DIR = "./"

# .h5 Keras Weight File Storage Directories:

## Mutliple Output Fully Conected Neural Network
H5_DIR_FULLYCONNECTED_MULTIPLE_RELU = "fullyconnected_multiple.h5"
H5_DIR_FULLYCONNECTED_MULTIPLE_ELU = "fullyconnected_multiple_elu.h5"

## Multiple Output Convolutional Networks
H5_DIR_COVNET_MULTIPLE_RELU = "covnet_multiple_running.h5"
H5_DIR_COVNET_MULTIPLE_ELU = "covnet_multiple_elu.h5"

## Single Output Convolution Networks
H5_DIR_COVNET_JUMP_SIGMA = "covnet_jump_sigma_running.h5"
H5_DIR_COVNET_JUMP_MU = "covnet_jump_mu_running.h5"
H5_DIR_COVNET_LAMBDA = "covnet_lambda_running.h5"

## Ensembling Network
H5_DIR_ENSEMBLING = "ff_ensembling_large.h5"

In [2]:
import numpy as np
import math
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pylab

from scipy import stats
from numpy import random as nrand

%matplotlib inline

# Optional:
import seaborn as sns
sns.set(color_codes=True)
sns.set_style(style='white')
sns.set(font_scale=1.6)

In [3]:
import tensorflow as tf
import keras
from keras.models import Model, Sequential, model_from_json, load_model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, LSTM, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D, Conv2D, Input
from keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as K
from keras.layers.core import Reshape

Using TensorFlow backend.


In [4]:
from Merton_Jump_Diffusion_SDE_Utilities import \
    ModelParameters, \
    random_model_params, \
    geometric_brownian_motion_jump_diffusion_log_returns

In [5]:
def test_set_model_params():
    return ModelParameters(
        all_time=2000,
        all_delta=0.00396825396,
        all_sigma = 0.1,
        gbm_mu = 0.05,
        jumps_sigma=0.07,
        jumps_mu=0.05,
        jumps_lamda=0.02)

def standard_convolutional_test_set(sample_size = 1000, dim = (40, 50)):
    test_set = []
    for i in range(sample_size):
        if (i%100 == 0):
            print(i)
        test_set.append(geometric_brownian_motion_jump_diffusion_log_returns(test_set_model_params()))

    test_set_pred = np.reshape(np.array(test_set), (sample_size, dim[0], dim[1], 1))
    return test_set_pred

def convolutional_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_data = np.reshape(np.array(RETURNS_), (paramset_size*paths_p_paramset, 40, 50, 1))
        sigmas = np.array([[mp.all_sigma] for mp in PARAMS_])
        mus = np.array([[mp.gbm_mu] for mp in PARAMS_])
        jump_sigmas = np.array([[mp.jumps_sigma] for mp in PARAMS_])
        jump_mus = np.array([[mp.jumps_mu] for mp in PARAMS_])
        lambdas = np.array([[mp.lamda] for mp in PARAMS_])
        
        yield train_data, [sigmas, mus, jump_sigmas, jump_mus, lambdas]

def standard_fullyconnected_test_set(sample_size = 1000):
    test_set = []
    for i in range(sample_size):
        if (i%100 == 0):
            print(i)
        test_set.append(geometric_brownian_motion_jump_diffusion_log_returns(test_set_model_params()))

    train_moments = [np.append(stats.moment(example, moment = range(2,20)), np.mean(example)) for example in test_set]
    train_statistics = np.array([np.append(train_moments[i], 
                                  sm.tsa.stattools.acf(test_set[i])) 
                        for i in range(len(test_set))])
    return train_statistics

def fullyconnected_batch_generator(all_time = 2000, paramset_size = 32, paths_p_paramset = 5):
    while True:
        RETURNS_ = []
        PARAMS_ = []
        
        for i in range(paramset_size):
            mp = random_model_params()
            mp.all_time = all_time
            
            for j in range(paths_p_paramset):
                PARAMS_.append(mp)
                RETURNS_.append(geometric_brownian_motion_jump_diffusion_log_returns(mp))
        
        train_moments = [np.append(stats.moment(example, moment = range(2,20)), np.mean(example)) for example in RETURNS_]

        train_statistics = np.array([np.append(train_moments[i], 
                              sm.tsa.stattools.acf(RETURNS_[i])) 
                    for i in range(len(RETURNS_))])
        
        sigmas = np.array([[mp.all_sigma] for mp in PARAMS_])
        mus = np.array([[mp.gbm_mu] for mp in PARAMS_])
        jump_sigmas = np.array([[mp.jumps_sigma] for mp in PARAMS_])
        jump_mus = np.array([[mp.jumps_mu] for mp in PARAMS_])
        lambdas = np.array([[mp.lamda] for mp in PARAMS_])
        
        yield train_statistics, [sigmas, mus, jump_sigmas, jump_mus, lambdas]

In [6]:
def r2(y_true, y_pred):
    SSE = K.sum(K.square(y_true-y_pred))
    SST = K.sum(K.square(y_true-K.mean(y_true)))
    return 1-SSE/SST

In [7]:
standard_fullyconnected_test_set()

0
100
200
300
400
500
600
700
800
900


array([[  1.28401948e-04,   1.25700915e-05,   2.18597808e-06, ...,
         -1.50446495e-03,  -5.59616860e-03,  -1.69203002e-02],
       [  2.01208391e-04,   1.86758234e-05,   2.89738079e-06, ...,
          2.60998158e-03,   3.93933525e-03,  -1.12003595e-02],
       [  1.77196660e-04,   1.62905935e-05,   2.47130273e-06, ...,
         -3.35692969e-02,  -2.49140978e-02,  -8.08779658e-03],
       ..., 
       [  1.93573397e-04,   2.11759088e-05,   3.60595244e-06, ...,
         -3.52877441e-02,   3.73902009e-02,  -8.61511139e-03],
       [  1.79082280e-04,   1.92471789e-05,   3.75637854e-06, ...,
         -1.00587505e-02,  -2.10471731e-02,  -3.39315339e-02],
       [  1.55189773e-04,   1.51306150e-05,   2.24810186e-06, ...,
          7.84744251e-03,   4.53653036e-03,   1.55394950e-02]])